# Inicializar

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import sqrt, exp, log, pi
from scipy import stats as st # calcular a moda de um array
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, f1_score

def k_fold(X, Y, k):
    n,m = X.shape
    ordenacao = np.arange(n)
    np.random.shuffle(ordenacao)
    saida = []
    intervalo = n // k
    grupos_X = []
    grupos_Y = []
    for i in range(k):
        grupos_X.append( X[ordenacao % k == i] )
        grupos_Y.append( Y[ordenacao % k == i] )

    return grupos_X, grupos_Y

def matriz_desvios(x, media):
    desvio = np.atleast_2d(x-media)
    matriz_dev = desvio.T @ desvio
    return matriz_dev
  
def gerar_mat_cov(X):
    n, p = X.shape
    media = np.mean(X, axis=0)
    vetor_matrizes = np.array([ matriz_desvios(X[i], media) for i in range(n)])
    return np.sum(vetor_matrizes, axis = 0) / (n-1)

def criar_acuracias(y_pred, y_real, K):
    y_pred = np.atleast_2d(y_pred)
    if y_pred.shape[1] == 1: y_pred = y_pred.T
    y_real = np.atleast_2d(y_real)
    if y_real.shape[1] == 1: y_real = y_real.T
    
    
    iguais = y_pred == y_real
    n = y_real.shape[1]
    acuracia_global = np.sum(iguais) / n
    acuracia_grupos = []
    for i in range(K):
        acuracia_grupos.append( np.sum( iguais[y_real == i]) / np.sum(y_real == i))
    return (acuracia_global, acuracia_grupos)
    

def dist_euclidiana(x1, x2):
    x1 = np.atleast_2d(x1)
    if x1.shape[1] == 1: x1 = x1.T
    x2 = np.atleast_2d(x2)
    if x2.shape[1] == 1: x2 = x2.T
    
    return  sqrt(np.sum((x1-x2)**2))

def dist_manhattan(x1, x2):
    x1 = np.atleast_2d(x1)
    if x1.shape[1] == 1: x1 = x1.T
    x2 = np.atleast_2d(x2)
    if x2.shape[1] == 1: x2 = x2.T
    
    return np.sum(np.abs(x1-x2))

def dist_mahalanobis(x1, x2, mat_cov):
    x1 = np.atleast_2d(x1)
    if x1.shape[1] == 1: x1 = x1.T
    x2 = np.atleast_2d(x2)
    if x2.shape[1] == 1: x2 = x2.T
    
    return sqrt( (x1-x2) @ mat_cov @ (x1-x2).T )  
    

# KNN

In [3]:
def KNN(X_treino, X_teste, y_treino, K = 1, distancia = "euclidiana", mat_cov = None):
    y_treino = np.atleast_2d(y_treino).astype(int)
    if y_treino.shape[0] == 1: y_treino = y_treino.T
    n, m = X_treino.shape
    K = min(K, n)
    
    map_distancias = {"euclidiana" : dist_euclidiana, "manhattan": dist_manhattan, "mahalanobis": dist_mahalanobis} 
    d = map_distancias[distancia]
    
    ordenacoes_teste = []
    for xi_teste in X_teste:
        ordenacao_aux = []
        for xi_treino in X_treino:
            if distancia == "mahalanobis":
                ordenacao_aux.append( d(xi_teste, xi_treino, mat_cov) )
            else:
                ordenacao_aux.append( d(xi_teste, xi_treino) )
        ordenacoes_teste.append( np.argsort(np.array(ordenacao_aux)) )
    ordenacoes_teste = np.array(ordenacoes_teste)
    
    escolhidos = y_treino[ ordenacoes_teste[:,:K] ]
    y_pred, frequencia = st.mode(escolhidos, axis = 1, keepdims = False)
    return y_pred.T

# Primeira Questão

In [7]:
dataset_q1 = np.genfromtxt('./kc2.csv', delimiter=',')
dataset_X = dataset_q1[:,:-1]
dataset_Y = dataset_q1[:,-1].astype(int)


grupos_X, grupos_Y = k_fold(dataset_X, dataset_Y, 10)
n_atribs = len(np.unique(dataset_Y))

# KNN
for Ki, f_distancia in ( (1, "euclidiana"), (5, "euclidiana"), (1, "mahalanobis"), (5, "mahalanobis") ):
    print("------------- "+str(Ki)+"-NN | "+ f_distancia +" -------------")
    array_acc_global = []
    array_acc_grupos = []
    array_revocacao = []
    array_precisao = []
    array_f1score = []
    for k in range(10):
        treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
        treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
        teste_X = grupos_X[k]
        teste_Y = grupos_Y[k]
        n = teste_X.shape[0]
        matriz_covariancia = gerar_mat_cov(treino_X)

        predicoes_Y = KNN(treino_X, teste_X, treino_Y, K = Ki, distancia = f_distancia, mat_cov = matriz_covariancia)

        acc_global, acc_grupos = criar_acuracias(predicoes_Y, teste_Y, n_atribs)
        revocacao = recall_score(teste_Y, predicoes_Y.T, average = None)
        precisao = precision_score(teste_Y, predicoes_Y.T, average=None)
        f1score = f1_score(teste_Y, predicoes_Y.T, average = None)
        
        array_acc_global.append(acc_global)
        array_acc_grupos.append(acc_grupos)
        array_revocacao.append(revocacao)
        array_precisao.append(precisao)
        array_f1score.append(f1score)

    media_acc_global = np.mean( np.array(array_acc_global) )
    dp_acc_global = np.std( np.array(array_acc_global) )

    media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
    dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)

    media_revocacao = np.mean( np.array(array_revocacao) , axis = 0)
    dp_revocacao = np.std( np.array(array_revocacao) , axis = 0)

    media_precisao = np.mean( np.array(array_precisao) , axis = 0)
    dp_precisao = np.std( np.array(array_precisao) , axis = 0)

    media_f1score = np.mean( np.array(array_f1score) , axis = 0)
    dp_f1score = np.std( np.array(array_f1score) , axis = 0)

    print("Valor médio acurácia global:", media_acc_global)
    print("Desvio Padrão acurácia global:", dp_acc_global)
    print()
    print("Valor médio acurácia por grupo:",media_acc_grupos)
    print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
    print()
    print("Valor médio revocação por grupo:",media_revocacao)
    print("Desvio Padrão revocação por grupo:", dp_revocacao)
    print()
    print("Valor médio precisão por grupo:",media_precisao)
    print("Desvio Padrão precisão por grupo:", dp_precisao)
    print()
    print("Valor médio f1_score por grupo:",media_f1score)
    print("Desvio Padrão f1_score por grupo:", dp_f1score)
    print()
    print()
    print()
    
    
# Arvore de decisão
print("------------- Arvore de decisão | Gini -------------")
array_acc_global = []
array_acc_grupos = []
array_revocacao = []
array_precisao = []
array_f1score = []
for k in range(10):
    treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
    treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
    teste_X = grupos_X[k]
    teste_Y = grupos_Y[k]
    n = teste_X.shape[0]

    dtc_gini = DecisionTreeClassifier(criterion='gini')
    dtc_gini.fit(treino_X, treino_Y)
    predicoes_Y = dtc_gini.predict(teste_X)
    
    

    acc_global, acc_grupos = criar_acuracias(predicoes_Y, teste_Y, n_atribs)
    revocacao = recall_score(teste_Y, predicoes_Y, average = None)
    precisao = precision_score(teste_Y, predicoes_Y, average=None)
    f1score = f1_score(teste_Y, predicoes_Y, average = None)
    
    array_acc_global.append(acc_global)
    array_acc_grupos.append(acc_grupos)
    array_revocacao.append(revocacao)
    array_precisao.append(precisao)
    array_f1score.append(f1score)

media_acc_global = np.mean( np.array(array_acc_global) )
dp_acc_global = np.std( np.array(array_acc_global) )

media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)

media_revocacao = np.mean( np.array(array_revocacao) , axis = 0)
dp_revocacao = np.std( np.array(array_revocacao) , axis = 0)

media_precisao = np.mean( np.array(array_precisao) , axis = 0)
dp_precisao = np.std( np.array(array_precisao) , axis = 0)

media_f1score = np.mean( np.array(array_f1score) , axis = 0)
dp_f1score = np.std( np.array(array_f1score) , axis = 0)

print("Valor médio acurácia global:", media_acc_global)
print("Desvio Padrão acurácia global:", dp_acc_global)
print()
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()
print("Valor médio revocação por grupo:",media_revocacao)
print("Desvio Padrão revocação por grupo:", dp_revocacao)
print()
print("Valor médio precisão por grupo:",media_precisao)
print("Desvio Padrão precisão por grupo:", dp_precisao)
print()
print("Valor médio f1_score por grupo:",media_f1score)
print("Desvio Padrão f1_score por grupo:", dp_f1score)
print()
print()
print()

print("------------- Arvore de decisão | Entropia -------------")
array_acc_global = []
array_acc_grupos = []
array_revocacao = []
array_precisao = []
array_f1score = []
for k in range(10):
    treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
    treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
    teste_X = grupos_X[k]
    teste_Y = grupos_Y[k]
    n = teste_X.shape[0]

    dtc_entropy = DecisionTreeClassifier(criterion='entropy')
    dtc_entropy.fit(treino_X, treino_Y)
    predicoes_Y = dtc_entropy.predict(teste_X)
    
    

    acc_global, acc_grupos = criar_acuracias(predicoes_Y, teste_Y, n_atribs)
    revocacao = recall_score(teste_Y, predicoes_Y, average = None)
    precisao = precision_score(teste_Y, predicoes_Y, average=None)
    f1score = f1_score(teste_Y, predicoes_Y, average = None)
    
    array_acc_global.append(acc_global)
    array_acc_grupos.append(acc_grupos)
    array_revocacao.append(revocacao)
    array_precisao.append(precisao)
    array_f1score.append(f1score)

media_acc_global = np.mean( np.array(array_acc_global) )
dp_acc_global = np.std( np.array(array_acc_global) )

media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)

media_revocacao = np.mean( np.array(array_revocacao) , axis = 0)
dp_revocacao = np.std( np.array(array_revocacao) , axis = 0)

media_precisao = np.mean( np.array(array_precisao) , axis = 0)
dp_precisao = np.std( np.array(array_precisao) , axis = 0)

media_f1score = np.mean( np.array(array_f1score) , axis = 0)
dp_f1score = np.std( np.array(array_f1score) , axis = 0)

print("Valor médio acurácia global:", media_acc_global)
print("Desvio Padrão acurácia global:", dp_acc_global)
print()
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()
print("Valor médio revocação por grupo:",media_revocacao)
print("Desvio Padrão revocação por grupo:", dp_revocacao)
print()
print("Valor médio precisão por grupo:",media_precisao)
print("Desvio Padrão precisão por grupo:", dp_precisao)
print()
print("Valor médio f1_score por grupo:",media_f1score)
print("Desvio Padrão f1_score por grupo:", dp_f1score)
print()
print()
print()

------------- 1-NN | euclidiana -------------
Valor médio acurácia global: 0.6216450216450217
Desvio Padrão acurácia global: 0.11448477351378927

Valor médio acurácia por grupo: [0.51904401 0.72133894]
Desvio Padrão acurácia por grupo: [0.22352414 0.1688515 ]

Valor médio revocação por grupo: [0.51904401 0.72133894]
Desvio Padrão revocação por grupo: [0.22352414 0.1688515 ]

Valor médio precisão por grupo: [0.66332307 0.61218281]
Desvio Padrão precisão por grupo: [0.24442225 0.10770294]

Valor médio f1_score por grupo: [0.55304492 0.65061688]
Desvio Padrão f1_score por grupo: [0.20324906 0.1026875 ]



------------- 5-NN | euclidiana -------------
Valor médio acurácia global: 0.7610389610389611
Desvio Padrão acurácia global: 0.05154242793467436

Valor médio acurácia por grupo: [0.79402958 0.73603452]
Desvio Padrão acurácia por grupo: [0.15402846 0.13877755]

Valor médio revocação por grupo: [0.79402958 0.73603452]
Desvio Padrão revocação por grupo: [0.15402846 0.13877755]

Valor médio 